# Fake News Classifier Using Bidirectional LSTM

Dataset: https://www.kaggle.com/c/fake-news/data#

#### Loading the libraries

In [1]:
import pandas as pd
from keras.layers import Embedding, LSTM, Bidirectional
from keras.layers import Dense, Dropout
from keras.models import Sequential
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import one_hot

#### Load the dataset

In [2]:
df = pd.read_csv('train.csv')
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


#### Drop NaN values

In [3]:
df = df.dropna()

#### Get Independent Features and Dependent Features

In [4]:
X = df.drop('label', axis = 1)
y = df['label']

In [5]:
X.shape

(18285, 4)

In [6]:
y.shape

(18285,)

In [7]:
y.value_counts()
# 7924 are fake news and 10361 are real news

0    10361
1     7924
Name: label, dtype: int64

In [8]:
voc_size = 5000

### OneHot Representation

In [9]:
import nltk
import re
from nltk.corpus import stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [10]:
text = X.copy()

In [11]:
text.head()

,id,title,author,text
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ..."
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...


In [12]:
text['title'][1]

'FLYNN: Hillary Clinton, Big Woman on Campus - Breitbart'

In [13]:
text.reset_index(inplace = True)

#### Text Preprocessing

In [14]:
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()

In [15]:
corpus = []
for i in range(0, len(text)):
    review = re.sub('[^a-zA-Z]', ' ', text['title'][i])
    review = review.lower()
    review = review.split()
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

In [16]:
len(corpus)

18285

In [17]:
corpus[:10]

['hous dem aid even see comey letter jason chaffetz tweet',
 'flynn hillari clinton big woman campu breitbart',
 'truth might get fire',
 'civilian kill singl us airstrik identifi',
 'iranian woman jail fiction unpublish stori woman stone death adulteri',
 'jacki mason hollywood would love trump bomb north korea lack tran bathroom exclus video breitbart',
 'beno hamon win french socialist parti presidenti nomin new york time',
 'back channel plan ukrain russia courtesi trump associ new york time',
 'obama organ action partner soro link indivis disrupt trump agenda',
 'bbc comedi sketch real housew isi caus outrag']

In [18]:
onehot_repr = [one_hot(words, voc_size) for words in corpus]
onehot_repr[:5]

[[4185, 4176, 3784, 4286, 1914, 4386, 2516, 4338, 3771, 4609],
 [1004, 248, 1926, 3789, 3994, 4073, 4164],
 [1917, 2166, 4175, 4660],
 [2647, 4321, 340, 1742, 3978, 3049],
 [3576, 3994, 1097, 2870, 3954, 4837, 3994, 2327, 3101, 2066]]

### Embedding Representation

In [19]:
sent_length = 50
embedded_docs = pad_sequences(onehot_repr, padding = 'pre', maxlen = sent_length)
print(embedded_docs)

[[   0    0    0 ... 4338 3771 4609]
 [   0    0    0 ... 3994 4073 4164]
 [   0    0    0 ... 2166 4175 4660]
 ...
 [   0    0    0 ... 2352 1252 3773]
 [   0    0    0 ... 4522 1558 3577]
 [   0    0    0 ... 2950 3121 2551]]


In [20]:
embedded_docs[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0, 4185, 4176, 3784, 4286,
       1914, 4386, 2516, 4338, 3771, 4609])

In [21]:
len(embedded_docs)

18285

### Train-Test Split

In [22]:
import numpy as np

In [23]:
X = np.array(embedded_docs)
y = np.array(y)

In [24]:
X.shape

(18285, 50)

In [25]:
y.shape

(18285,)

In [26]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 42)

### Creating the Model

In [27]:
embedding_vector_features = 40
model = Sequential()
model.add(Embedding(voc_size, embedding_vector_features, input_length = sent_length))
model.add(Bidirectional(LSTM(128)))
model.add(Dropout(0.3))
model.add(Dense(1, activation = 'sigmoid'))
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 50, 40)            200000    
_________________________________________________________________
bidirectional (Bidirectional (None, 256)               173056    
_________________________________________________________________
dropout (Dropout)            (None, 256)               0         
_________________________________________________________________
dense (Dense)                (None, 1)                 257       
Total params: 373,313
Trainable params: 373,313
Non-trainable params: 0
_________________________________________________________________


In [28]:
model.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])

### Model Training

In [29]:
model.fit(X_train, y_train, validation_data = (X_test, y_test), epochs = 10,batch_size = 64)

Epoch 1/10
200/200 [==============================] - 12s 51ms/step - loss: 0.4788 - accuracy: 0.7428 - val_loss: 0.1926 - val_accuracy: 0.9202
Epoch 2/10
200/200 [==============================] - 9s 45ms/step - loss: 0.1407 - accuracy: 0.9453 - val_loss: 0.1973 - val_accuracy: 0.9231
Epoch 3/10
200/200 [==============================] - 8s 42ms/step - loss: 0.0890 - accuracy: 0.9679 - val_loss: 0.2441 - val_accuracy: 0.9172
Epoch 4/10
200/200 [==============================] - 9s 43ms/step - loss: 0.0642 - accuracy: 0.9805 - val_loss: 0.2610 - val_accuracy: 0.9136
Epoch 5/10
200/200 [==============================] - 9s 43ms/step - loss: 0.0488 - accuracy: 0.9832 - val_loss: 0.3062 - val_accuracy: 0.9012
Epoch 6/10
200/200 [==============================] - 9s 46ms/step - loss: 0.0408 - accuracy: 0.9870 - val_loss: 0.3807 - val_accuracy: 0.9007
Epoch 7/10
200/200 [==============================] - 9s 44ms/step - loss: 0.0283 - accuracy: 0.9910 - val_loss: 0.3922 - val_accuracy: 0.911

### Performance Metrics and Accuracy

In [30]:
y_pred = model.predict_classes(X_test)

C:\Users\User\anaconda3\envs\gpu_tf\lib\site-packages\tensorflow\python\keras\engine\sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


In [31]:
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

In [32]:
confusion_matrix(y_test, y_pred)

array([[2822,  285],
       [ 229, 2150]], dtype=int64)

In [33]:
accuracy_score(y_test, y_pred)

0.9063069631790011

In [34]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.92      0.91      0.92      3107
           1       0.88      0.90      0.89      2379

    accuracy                           0.91      5486
   macro avg       0.90      0.91      0.90      5486
weighted avg       0.91      0.91      0.91      5486

